In [5]:
import requests
import os
import json
import config
import preprocessor as p #cleaner
from langdetect import detect #language detecter
from csv import writer #write to a csv file





# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def get_rules(headers, bearer_token):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", headers=headers
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()


def delete_all_rules(headers, bearer_token, rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers = headers,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))


def set_rules(headers,delete,bearer_token):
    # You can adjust the rules if needed
    sample_rules = [
        {"value": "appl", "tag": "appl"},
       # {"value": "cat has:images -grumpy", "tag": "cat pictures"},
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        headers=headers,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))


def get_stream(headers,set,bearer_token):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", headers=headers, stream=True,
    )
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            #print(json.dumps(json_response, indent=4, sort_keys=True))
            tweet = json_response['data']['text']
            tweet = p.clean(tweet) #cleaner
            #print(tweet)
            try: #compatible tweets with detect
                if detect(tweet) == 'en':
                    print(tweet)
                    tweetlst = [tweet]
            except:
                pass

def main():
    bearer_token = config.BEARER_TOKEN
    headers = create_headers(bearer_token)
    rules = get_rules(headers, bearer_token)
    delete = delete_all_rules(headers, bearer_token, rules)
    set = set_rules(headers, delete, bearer_token)
    get_stream(headers, set, bearer_token)

if __name__ == "__main__":
    main()


{"data": [{"id": "1520256008483921920", "value": "appl", "tag": "appl"}], "meta": {"sent": "2022-04-30T04:33:19.696Z", "result_count": 1}}
{"meta": {"sent": "2022-04-30T04:33:21.049Z", "summary": {"deleted": 1, "not_deleted": 0}}}
{"data": [{"value": "appl", "tag": "appl", "id": "1520259989692198912"}], "meta": {"sent": "2022-04-30T04:33:22.687Z", "summary": {"created": 1, "not_created": 0, "valid": 1, "invalid": 0}}}
200


KeyboardInterrupt: 

In [46]:
from ernie import SentenceClassifier, Models
import pandas as pd


df = pd.read_csv("dataset.csv")
print(df)

                                        Sentiment Text Sentiment
0    : has now transfered $4,000,000,000,000 around...   Bullish
1    Bitcoin next move....? In the next comming month.   Bullish
2    If you have a phone or a computer, why not let...   Neutral
3    Pump Alert! - UTK/USDT on Binance! Price incre...   Neutral
4    Crypto Pump Alert! - Frankie WSB (cancelled) s...   Neutral
..                                                 ...       ...
244                        get out before they doom us   Bearish
245                        help me its too late for me   Bearish
246                         APPL giving me a headache    Bearish
247                     the earnings are out tomorrow    Neutral
248               pray for me, help me out of this pit   Bearish

[249 rows x 2 columns]


In [47]:
mapper = {'Bearish': 0, 'Neutral': 1, 'Bullish': 2}
df.Sentiment = df.Sentiment.map(mapper)

#maps the sentiment to numbers 



In [48]:
print (df)
df.Sentiment.value_counts()
df.columns = ['SentimentText','Sentiment']

                                        Sentiment Text  Sentiment
0    : has now transfered $4,000,000,000,000 around...          2
1    Bitcoin next move....? In the next comming month.          2
2    If you have a phone or a computer, why not let...          1
3    Pump Alert! - UTK/USDT on Binance! Price incre...          1
4    Crypto Pump Alert! - Frankie WSB (cancelled) s...          1
..                                                 ...        ...
244                        get out before they doom us          0
245                        help me its too late for me          0
246                         APPL giving me a headache           0
247                     the earnings are out tomorrow           1
248               pray for me, help me out of this pit          0

[249 rows x 2 columns]


In [49]:
#clean and shuffle data
df.SentimentText = df.SentimentText.str.replace(':',"")
#replace colons from tweets added from stream
df = df.sample(frac=1).reset_index(drop=True)#randomise


In [50]:
print(df)

                                         SentimentText  Sentiment
0    is booming Will it last, or will it bust? via ...          0
1     THE CRYPTO GIVEAWAY CONTEST IS NOW LIVE! $25K...          1
2     $ARDR is a multichain platform with a parent-...          1
3         An alternate way to look at price over time.          1
4    Earn a share of million $REAP cryptocurrency i...          1
..                                                 ...        ...
244  Sorry for bothering you. But I actually have a...          1
245               closed above $50k for days in a row!          2
246                    Still got my eyes on the prize.          2
247   New Listing will list - Trading pair DRYCAKE/...          1
248  Bitcoin next move....? In the next comming month.          2

[249 rows x 2 columns]


In [51]:
#import classifier
classifier = SentenceClassifier(model_name='bert-base-uncased', max_length=64, labels_no=3)
classifier.load_dataset(df, validation_split=0.1)
classifier.fine_tune(epochs=10, learning_rate=2e-5)


In [53]:
#dataset into a format classifier can understand
classifier.load_dataset(df, validation_split=0.1)
#train on 90% of the data and test on 10%

In [55]:
#fine tune classifier
print("training")
classifier.fine_tune(epochs=10, learning_rate=2e-5) #learning rate from documentation

training
7/7 [==============================] - ETA: 0s - loss: 0.1734 - accuracy: 0.9688WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 25 batches). You may need to use the repeat() function when building your dataset.


2022-04-30 03:49:24 [WARNING] Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 25 batches). You may need to use the repeat() function when building your dataset.


7/7 [==============================] - ETA: 0s - loss: 0.1463 - accuracy: 0.9598WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 25 batches). You may need to use the repeat() function when building your dataset.


2022-04-30 03:49:50 [WARNING] Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 25 batches). You may need to use the repeat() function when building your dataset.


7/7 [==============================] - ETA: 0s - loss: 0.1297 - accuracy: 0.9643WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 25 batches). You may need to use the repeat() function when building your dataset.


...

2022-04-30 03:51:39 [WARNING] Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 25 batches). You may need to use the repeat() function when building your dataset.


7/7 [==============================] - ETA: 0s - loss: 0.0819 - accuracy: 0.9688WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 25 batches). You may need to use the repeat() function when building your dataset.


2022-04-30 03:52:06 [WARNING] Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 25 batches). You may need to use the repeat() function when building your dataset.


3/7 [===========>..................] - ETA: 14s - loss: 0.0509 - accuracy: 0.9896

KeyboardInterrupt: 

In [56]:
#loss to go down and accurray to go up


print(df)

                                         SentimentText  Sentiment
0    is booming Will it last, or will it bust? via ...          0
1     THE CRYPTO GIVEAWAY CONTEST IS NOW LIVE! $25K...          1
2     $ARDR is a multichain platform with a parent-...          1
3         An alternate way to look at price over time.          1
4    Earn a share of million $REAP cryptocurrency i...          1
..                                                 ...        ...
244  Sorry for bothering you. But I actually have a...          1
245               closed above $50k for days in a row!          2
246                    Still got my eyes on the prize.          2
247   New Listing will list - Trading pair DRYCAKE/...          1
248  Bitcoin next move....? In the next comming month.          2

[249 rows x 2 columns]


In [63]:
import numpy as np

text = "MSFT earnings are out tomorrow"

print(text)
print("========================================")
probabilities = classifier.predict_one(text)
print(probabilities)
classes = ["Bearish", "Neutral", "Bullish"]
print("========================================")
print(classes[np.argmax(probabilities)])

MSFT earnings are out tomorrow
(0.011232829939373159, 0.9774283373854925, 0.011338832675134369)
Neutral


In [58]:
text = "The big day has come! This Sunday we are launching our own amazing NFT platform! TO THE MOOOON!!"

print(text)
print("========================================")
probabilities = classifier.predict_one(text)
print(probabilities)
classes = ["Bearish", "Neutral", "Bullish"]
print("========================================")
print(classes[np.argmax(probabilities)])

The big day has come! This Sunday we are launching our own amazing NFT platform! TO THE MOOOON!!
(0.006389703190386593, 0.26059921186461965, 0.7330110849449938)
Bullish


In [72]:
text = "Neither a rich man or a poor man gets to enjoy his money. One is too busy trying to hold onto it, and the other is too busy trying to get it . Live laugh & love as much as you can 💗"

print(text)
print("========================================")
probabilities = classifier.predict_one(text)
print(probabilities)
classes = ["Bearish", "Neutral", "Bullish"]
print("========================================")
print(classes[np.argmax(probabilities)])

Neither a rich man or a poor man gets to enjoy his money. One is too busy trying to hold onto it, and the other is too busy trying to get it . Live laugh & love as much as you can 💗
(0.00313096557735057, 0.8702835638637582, 0.1265854705588911)
Neutral


In [79]:
text = "Markets may rally first to make the stage for a big gains."

print(text)
print("========================================")
probabilities = classifier.predict_one(text)
print(probabilities)
classes = ["Bearish", "Neutral", "Bullish"]
print("========================================")
print(classes[np.argmax(probabilities)])

Markets may rally first to make the stage for a big gains.
(0.012845361417848634, 0.015881746245201476, 0.9712728923369499)
Bullish


In [82]:
#trigger launch script to stream tweets from django
#timer using try and except
#append the tweets/semantic analysis to a array
#display on html page.

hello


In [4]:
import time

t_end = time.time() + 10
while time.time() < t_end:
    bearer_token = config.BEARER_TOKEN
    headers = create_headers(bearer_token)
    rules = get_rules(headers, bearer_token)
    delete = delete_all_rules(headers, bearer_token, rules)
    set = set_rules(headers, delete, bearer_token)
    get_stream(headers, set, bearer_token)
print("ended")


NameError: name 'create_headers' is not defined

In [20]:
def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers, stream=True)
    print(response.status_code)
    for response_line in response.iter_lines():
        if response_line:
            json_response = json.loads(response_line)
            print(json.dumps(json_response, indent=4, sort_keys=True))
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    
    
def main():
    print("here")
    url = create_url()
    bearer_token = config.BEARER_TOKEN
    headers = create_headers(bearer_token)
    rules = get_rules(headers, bearer_token)
    delete = delete_all_rules(headers, bearer_token, rules)
    set = set_rules(headers, delete, bearer_token)
    timeout = 0
    while True:
        connect_to_endpoint(url, headers)
        print("connected")
        timeout += 1